In [ ]:
import logging
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from lmflux import (
    openai_agent, tool,
    Session, Agent, Message
)
from lmflux.graphs import transformer_task, agentic_task, TaskGraph

# Creating a new tool
@tool
def add(a: int, b: int) -> int:
    """Adds two numbes together"""
    return {"result": a + b}

# Create an agent
agent_oss = openai_agent("my-agent", "openai.gpt_oss_20b", tools=[add])

#Tasks
@transformer_task
def set_data(session: Session):
    session.set("a", 985)
    session.set("b", 1265)

@agentic_task(agent_oss)
def add_task(agent:Agent, session:Session):
    # This resets the internal conversation if there is any
    agent.reset_agent_state()
    # Builds the template
    a = session.get("a")
    b = session.get("b")
    message = Message(
        "user", f"What is {a}+{b}?"
    )
    response = agent.conversate(message, session)
    session.set("result", response.content)

G = TaskGraph()
G.connect_tasks(set_data, add_task)
G.show_mermaid()

```mermaid
graph TB
9b804ed4-64db-43f6-b69f-94411f773c98(start)
52977173-a5f2-4eb2-bb16-c7626c8e2a8b(end)
style 9b804ed4-64db-43f6-b69f-94411f773c98 fill:#ffcc00,stroke:#333,stroke-width:2px,color:#000
style 52977173-a5f2-4eb2-bb16-c7626c8e2a8b fill:#ffcc00,stroke:#333,stroke-width:2px,color:#000
28e7726e-d16d-4d1f-b499-283658471b09(set_data)
ea7a1280-3a5f-4323-a305-b697eb8e2776(add_task)
28e7726e-d16d-4d1f-b499-283658471b09 --> ea7a1280-3a5f-4323-a305-b697eb8e2776
9b804ed4-64db-43f6-b69f-94411f773c98 --> 28e7726e-d16d-4d1f-b499-283658471b09
ea7a1280-3a5f-4323-a305-b697eb8e2776 --> 52977173-a5f2-4eb2-bb16-c7626c8e2a8b
```

In [3]:
session = G.run()
session.context.get('result')

'The sum of 985 and 1265 is **2250**.'

In [7]:
add_task.agent.llm.conversation

Conversation:
----
Message(system):
	content: You are a helpful assistant.
Message(user):
	content: What is 985+1265?
Message(assistant):
	tool call: [{'id': 'functions.add.add', 'type': 'function', 'function': {'name': 'add', 'arguments': '{"a":985,"b":1265}'}}]
-----
	reasoning: The user asks: "What is 985+1265?" This is a simple addition question. The user expects the result. 985 + 1265 = 2250. Provide answer: 2250. Then we can use the provided function maybe. There's a function "add" that adds two numbers. We can call that. The user wants result. They didn't specifically request the function usage, but we can comply. But we should verify that we want to send the function call in the function format. The tool function signature: functions.add with arguments a and b. We'll call to produce result. Then we need to respond with the result. Since we are using function, we should produce a function execution. The system might want to see the function call. Yes, we can inline the function 